# Collect Physicists Raw Data

Here I collect biographical data on the list of [physicists notable for their achievements](../data/raw/physicists.txt). Wikipedia contains this structured data in an *Infobox* on the top right side of the article for each physicist. However, similar data is available in a more machine readable, *JSON* format from [DBpedia](https://wiki.dbpedia.org/about). For an example, compare [Albert Einstein's Wikipedia infobox](https://en.wikipedia.org/wiki/Albert_Einstein) to [Albert Einstein's DBPedia JSON](http://dbpedia.org/data/Albert_Einstein.json). It is important to note that the data is similar but not identical.

The shortcomings of Wikipedia infoboxes and the advantages of DBpedia datasets are explained in [DBpedia datasets (section 4.3)]. (https://wiki.dbpedia.org/services-resources/datasets/dbpedia-datasets). But basically the DBpedia data is much cleaner and better structured Wikipedia Infoboxes since it is based on hand-generated mappings of Wikipedia infoboxes/templates to a [DBpedia ontology](https://wiki.dbpedia.org/services-resources/ontology). As a result I choose DBpedia as the data source for this project.

However, dBpedia does have the disadvantage that its content is roughly 6-18 months behind updates applied to Wikipedia content. This is because its data is generated from a [static dump of Wikipedia content](https://wiki.dbpedia.org/online-access/DBpediaLive) in a process that takes approximately 6 months. The fact that the data is not in sync with the latest Wikipedia content is not so significant for this project as the data is edited infrequently and when it is there are only minor changes.

I will need to send HTTP requests to DBpedia to get the biographical JSON data on each of the physicists.

## Constructing the URLs

To make the HTTP requests, I will need a list of URLs representing the resources (physicists). It's fairly easy to construct these URLs from the list of notable physicists. However, it's important to `quote` any physicist name in unicode since unicode characters are not allowed in URLs. OK let's create the list now.

In [ ]:
import gzip
import os
import shutil
import time
from urllib import parse

import jsonlines
import pandas as pd
import progressbar as pb
import requests

In [ ]:
def construct_urls(file='../data/raw/physicists.txt'):
    """Construct DBpedia data URLs from list in file.

    Args:
        file (str): File containing a list of url filepaths
            with spaces replacing underscores.
    Returns:
        list(str): List of URLs.

    """

    with open(file, encoding='utf-8') as file:
        names = [line.rstrip('\n') for line in file]

    urls = []
    for name in names:
        url = name.replace(' ', '_')
        # some physicist names contain unicode characters
        # which have to be quoted when in a url
        if not all(ord(char) < 128 for char in name):
            url = parse.quote(url)
        url = 'http://dbpedia.org/data/' + url + '.json'
        urls.append(url)
    return urls

In [ ]:
urls = construct_urls()
assert(len(urls) == 1060)

## Fetching the Data

Now I have the list of URLs, it's time to make the HTTP requests to acquire the data. This will take a bit of time as there are 1059 urls and I want to crawl responsibly so that I don't bombard the site. Time to get a coffee.

In [ ]:
def fetch_json_data(urls, progress_bar=None):
    """Fetch json data from DBpedia.

    Args:
        urls (list(str)): List of URLs to fetch data from.
        progress_bar (progressbar.ProgressBar): Progress bar.
    Returns:
        list(json): List of JSON objects.

    """

    json_data = []
    for i in range(len(urls)):
        time.sleep(1)  # delay to crawl responsibly
        response = requests.get(urls[i])
        response.raise_for_status()
        if response.status_code == requests.codes.ok:
            json_data.append(response.json())
        if progress_bar:
            progress_bar.update(i)
    return json_data

In [ ]:
widgets = [
    'Fetching: ', pb.Counter(),
    ' / ' + str(len(urls)) + ' urls',
    ' ', pb.Bar(marker='█'),
    ' ', pb.Percentage(),
    ' ', pb.Timer(),
    ' ', pb.ETA()
]
bar = pb.ProgressBar(max_value=len(urls), widgets=widgets,
                     redirect_stdout=True).start()

json_data = fetch_json_data(urls, bar)
bar.finish()

Let's confirm that all the data was fetched and take a look at the first JSON response.

In [ ]:
assert(len(json_data) == 1060)
json_data[0]

It is clear that every request successfully received a response. However, I see that some responses came back empty from the server. Basically, although there are Wikipedia pages for these physicists they do not have a corresponding page in DBpedia (or the page in DBpedia has a different name). Not to worry, there are only 10 and they are not so famous, so I'll just exlude these physicists from the analysis.

In [ ]:
dropped_urls = [urls[i] for (i, url) in enumerate(json_data)
                if not url]
assert(len(dropped_urls) == 10)
print(len(dropped_urls))
dropped_urls

## Persisting the Data

Now I have the list of JSON responses, I'd like to persist them for later analysis. I'll use [Json Lines](http://jsonlines.org/) as it seems like a convenient format for storing structured data that may be processed one record at a time.

In [ ]:
jsonl_file = '../data/raw/notable_physicists.jsonl'
with jsonlines.open(jsonl_file, 'w') as writer:
    writer.write_all(filter(None, json_data))

Let's do a quick sanity check to make sure the file contains the expected number of records.

In [ ]:
json_lines = []
with jsonlines.open(jsonl_file, 'r') as reader:
    for json_line in reader:
        json_lines.append(json_line)
assert(len(json_lines) == 1050)

Finally, let's compress the file to reduce its footprint.

In [ ]:
with open(jsonl_file, 'rb') as src, gzip.open(jsonl_file + '.gz', 'wb') as dest:
    shutil.copyfileobj(src, dest)
os.remove(jsonl_file)

## Cleaning Up

A few clean up steps are needed:

- Convert the notebook to a HTML file with all the output.
- Convert the notebook to another notebook with the output removed.

In [ ]:
!jupyter nbconvert --output-dir html_output --to html 1.1-collect-physicists-raw-data.ipynb

In [ ]:
!jupyter nbconvert --ClearOutputPreprocessor.enabled=True --to notebook 1.1-collect-physicists-raw-data.ipynb